In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
cd ../..

/Users/zhixingzhang/Just for FUN/covid19-us-spread


In [56]:
df = pd.read_csv('data/cleaned/data_complete.csv')
cunts = df[['State', 'County', 'Population', 'Total Area (km)', 
            'Latitude','Longitude']].drop_duplicates().reset_index(drop=True)
cmmt = df[['State', 'County', 'Work State', 'Work County', 
        'Commuting Flow','Margin of Error']].drop_duplicates().set_index('State', drop=True)
virus_df = pd.read_csv('data/cleaned/virus.csv', parse_dates=['Date'])

In [37]:
virus_cty = virus_df.groupby(['State', 'County']).sum().loc[:, ['Case Count']]

In [57]:
cunts = pd.merge(cunts, virus_cty, how='left', left_on=['State', 'County'], right_on=['State', 'County'])
cunts.fillna(0, inplace=True)
cunts['Case Count'] = cunts['Case Count'].astype(int)

In [80]:
fig = go.Figure(data=go.Scattergeo(
        lon = cunts['Longitude'],
        lat = cunts['Latitude'],
        text = cunts['County'] + ', Population: ' + cunts['Population'].astype(str),
        mode = 'markers',
        marker_color = np.log(cunts.Population),
    ))

fig.update_layout(
    title = 'Population of US Counties',
    geo_scope='usa')

fig = go.Figure(data=go.Scattergeo(
        lon = cunts['Longitude'],
        lat = cunts['Latitude'],
        text = cunts['County'] + ', Population: ' + cunts['Population'].astype(str),
        mode = 'markers',
        marker_color = np.log(cunts.Population / cunts['Total Area (km)']),
    ))

fig.update_layout(
    title = 'Population Density of US Counties',
    geo_scope='usa')

fig = go.Figure(data=go.Scattergeo(
        lon = cunts['Longitude'],
        lat = cunts['Latitude'],
        text = cunts['County'] + \
            ', Population: ' + cunts['Population'].astype(str) + \
            ', Confirmed Cases: ' + cunts['Case Count'].astype(str),
        mode = 'markers',
        # marker_size=5,
        marker_color = np.log(cunts['Case Count'] / cunts['Population']),
    ))

fig.update_layout(
    title = 'Confirmed Cases Count over Population of US Counties',
    geo_scope='usa')